#Configure Credentials and access points
Some vary by user, some by cloud vs. on prem some by cloud region. Please adjust as needed

In [1]:
import os
PROJECT_UID= os.environ['PROJECT_ID'] #this assumes we are running in Studio. When running externally please add the project id here.
CPD_URL="https://services-uscentral.skytap.com:20623" #the variable starts with https://api." and we only need the rest 'dataplatform.dev.cloud.ibm.com'
CONTAINER_ID=PROJECT_UID
CONTAINER_TYPE='project'
EXPERIMENT_NAME='SciKitExperiment_fact'
print(CPD_URL)
#WML_URL='https://us-south.ml.cloud.ibm.com' #if you are running WML in other regions than Dallas you may have to update this.
# CPD4D API key. Navigate to hamburger menu -> Acess (IAM) -> API Keys to create one for you (or have your account admin do it)
#API_KEY='Add your API key here' 
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
#PROJECT_ACCESS_TOKEN='add your token here' #You can create or lookup such a token from the settings tab of this project

https://services-uscentral.skytap.com:20623


In [2]:
from project_lib import Project
project = Project(project_id=PROJECT_UID)

#Import & initialize IBM AI Governance Python library & client

In [3]:
!pip uninstall -y ibm-aigov-facts-client
!pip install --extra-index-url https://test.pypi.org/simple/ ibm-aigov-facts-client

Looking in indexes: https://pypi.org/simple, https://test.pypi.org/simple/
     |████████████████████████████████| 98 kB 2.1 MB/s eta 0:00:011
     |████████████████████████████████| 42 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 547 kB 50.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 44.9 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 181 kB 75.8 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 1.6 MB/s  eta 0:00:01
  Created wheel for databricks-cli: filename=databricks_cli-0.17.0-py3-none-any.whl size=141959 sha256=f13b2c21881e3fb3dc6b97dc0e13aa5a3f1c69b7660214b081e0814e3c45e9f5
  Stored in directory: /tmp/1000630000/.cache/pip/wheels/d5/b6/71/c3052c82e4a88dc658dd2616b944e130c1d0ff3f77e8f02df7
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5028 sha256=dcf44179ba58431921e051449fe5a750e1988b96ff7e217ad2fce8b

In [19]:
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig
creds=CloudPakforDataConfig(service_url="https://ibm-nginx-svc:443",
    username="admin",
    password="CP4DDataFabric")
facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

2022/07/13 16:16:32 INFO : Successfully set SciKitExperiment_fact as current experiment
2022/07/13 16:16:32 INFO : Autolog enabled Successfully


#Import  & initialize IBM WML library
Non-IBM runtimes are supported by AI Governance as well

In [5]:
!pip install ibm-watson-machine-learning

In [8]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                   "url": "https://ibm-nginx-svc:443",
                   "username": "admin",
                   "password" : "CP4DDataFabric",
                   "instance_id": "wml_local",
                   "version" : "4.5" #If your env is CP4D 4.0 then specify "4.0" instead of "3.5"
                  }
wml_client = APIClient(wml_credentials)
wml_client.set.default_project(PROJECT_UID)

'SUCCESS'

In [9]:
#Look up the data asset id corresponding to the data asset name of our training and test data
import sys
TRAINING_DATA_ASSSET_NAME = 'banking_demo_train.csv'
TEST_DATA_ASSSET_NAME = 'banking_demo_test.csv'
asset_details = wml_client.data_assets.get_details()
training_data_asset_id = ''
test_data_asset_id = ''
for data_asset in asset_details["resources"]:
    if data_asset["metadata"]["name"] == TRAINING_DATA_ASSSET_NAME:
          training_data_asset_id = data_asset["metadata"]["asset_id"]    
    if data_asset["metadata"]["name"] == TEST_DATA_ASSSET_NAME:
          test_data_asset_id = data_asset["metadata"]["asset_id"]    

if training_data_asset_id == '' :
    sys.exit('Error: No training data asset with name ' + TRAINING_DATA_ASSSET_NAME + ' found in this project.')
if test_data_asset_id == '' :
    sys.exit('Error: No test data asset with name ' + TEST_DATA_ASSSET_NAME + ' found in this project.') 

In [10]:
training_data_asset_url = "https://" + CPD_URL + "/projects/" + PROJECT_UID + "/data-assets/"+ training_data_asset_id + "/preview?context=cpdaas"
print(training_data_asset_url)
test_data_asset_url = "https://" + CPD_URL + "/projects/" + PROJECT_UID + "/data-assets/"+ test_data_asset_id + "/preview?context=cpdaas"
print(test_data_asset_url)

https://https://services-uscentral.skytap.com:20623/projects/6d8022e5-71c7-41c7-a697-e6c26fabebeb/data-assets/104e0dda-d5ff-4edd-a4b3-bdf8fdb34dbb/preview?context=cpdaas
https://https://services-uscentral.skytap.com:20623/projects/6d8022e5-71c7-41c7-a697-e6c26fabebeb/data-assets/1ab21305-2f54-48a3-bb86-a891c83c9d9d/preview?context=cpdaas


#Main section - regular ML development
The example below uses Scikit-based learning of credit risk - but really anything can be done here.
No IBM code needs to be added to this main section - but AI Gov will still capture facts from there.

In [11]:
# Fetch the file content of the training and test data assset using Watson Studio project lib 
import pandas as pd
train_file = project.get_file(TRAINING_DATA_ASSSET_NAME)
train_file.seek(0)
train_data = pd.read_csv(train_file)
#train_data.head()

test_file = project.get_file(TEST_DATA_ASSSET_NAME)
test_file.seek(0)
test_data = pd.read_csv(test_file)
#test_data.head()

In [12]:
MODEL_NAME = "Credit Risk Demo Model1 (SciKit)"
DEPLOYMENT_NAME = MODEL_NAME + " - Deployment"

In [13]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [14]:
#from sklearn.model_selection import train_test_split
#train_data, test_data = train_test_split(input_data, test_size=0.33)

In [15]:
import numpy as np
features_idx = np.s_[0:-1]
all_records_idx = np.s_[:]
first_record_idx = np.s_[0]

In [16]:
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(handle_unknown='ignore'), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log', penalty='l2', max_iter=1000, tol=1e-5)

pipeline = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

In [20]:
model = pipeline.fit(train_data.drop('Risk', axis=1), train_data.Risk)

2022/07/13 16:16:47 INFO : logging results to factsheet for run_id 366f6be8e7e544909af699a5455a5236
2022/07/13 16:16:48 INFO : Successfully logged results to Factsheet service for run_id 366f6be8e7e544909af699a5455a5236 under asset_id: 3a652a16-5ee3-46ec-b5e0-a5458742c1f6 and project_id : 6d8022e5-71c7-41c7-a697-e6c26fabebeb


In [21]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction=='No Risk' else 1 for prediction in predictions]

real_observations = test_data.Risk.replace('Risk', 1)
real_observations = real_observations.replace('No Risk', 0).values

auc = roc_auc_score(real_observations, indexed_preds)
print(auc)

0.7236293179406952


# IBM WML code (defining model metadata)

In [22]:
import numpy
def np2schema(dt):
    if dt == numpy.float64 : return "double"
    if dt == numpy.datetime64: return "date"
    if dt == numpy.int64 : return "integer"
    if dt == numpy.object: return "string"
    return "string"

def computeSchemasFromDf(data_frame, pred_column, pred_column_values):
    training_data_schema_fields = {"fields": []}
    input_schema_fields = {"fields": []}
    output_schema_fields = {"fields": [
            {
                'metadata': {
                    'modeling_role': 'probability'
                },
                'name': 'probability',
                'type': '[double]'
            },
            {
                'name': 'prediction',
                'type': 'double'
            },
            {
                'metadata': {
                    'modeling_role': 'prediction',
                    'values': pred_column_values
                },
                'name': 'predictedLabel',
                'type': 'string'
            }        
        ]
    }

    for column in data_frame:
        col = data_frame[column]
        field = {
            "metadata": {"modeling_role": "target"} if col.name == pred_column else {"modeling_role": "feature"},
            "nullable": True,
            "name": col.name,
            "type": np2schema(col.dtype)
        }
        infield = {
            "name": col.name,
            "type": np2schema(col.dtype)
        }
        training_data_schema_fields["fields"].append(infield)
        if col.name != pred_column : #we don't expect the label column in the input
            input_schema_fields["fields"].append(infield)
        if col.dtype == np.object : #mark strings a discrete measures
            field["metadata"]["measure"] ='discrete' 
        if col.name != pred_column : #the label column is separate and named "predictedLabel" in the output
            output_schema_fields["fields"].append(field)

    input_data_schema = [
        {
            'id': 'in_1',
            'fields': input_schema_fields["fields"]
        }
    ]

    output_data_schema = {
        'id': 'out',
        'fields': output_schema_fields["fields"]
    }

    training_data_schema = {
        "id": "train",
        "fields": training_data_schema_fields["fields"]
    }
    return input_data_schema, output_data_schema, training_data_schema


input_data_schema, output_data_schema, training_data_schema = computeSchemasFromDf(train_data, "Risk", ['No Risk', 'Risk'])
#import json
#print(json.dumps(output_data_schema, indent=4))


/tmp/1000630000/ipykernel_128/407650650.py:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if dt == numpy.object: return "string"
/tmp/1000630000/ipykernel_128/407650650.py:50: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if col.dtype == np.object : #mark strings a discrete measures


In [23]:
training_data_references = [
    {
        "id": TRAINING_DATA_ASSSET_NAME, 
        "type": "data_asset",
        "connection": {},
        "location": {
            "id": training_data_asset_id,
            "path": training_data_asset_url
        },
        "schema": training_data_schema
    }
]

In [24]:
wml_client._models.ConfigurationMetaNames.get()

['CUSTOM',
 'DESCRIPTION',
 'DOMAIN',
 'HYPER_PARAMETERS',
 'IMPORT',
 'INPUT_DATA_SCHEMA',
 'LABEL_FIELD',
 'METRICS',
 'MODEL_DEFINITION_UID',
 'NAME',
 'OUTPUT_DATA_SCHEMA',
 'PIPELINE_UID',
 'RUNTIME_UID',
 'SIZE',
 'SOFTWARE_SPEC_UID',
 'TAGS',
 'TEST_DATA_REFERENCES',
 'TF_MODEL_PARAMS',
 'TRAINING_DATA_REFERENCES',
 'TRAINING_LIB_UID',
 'TRANSFORMED_LABEL_FIELD',
 'TYPE']

In [25]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.0",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
        wml_client._models.ConfigurationMetaNames.INPUT_DATA_SCHEMA: input_data_schema
}

Software Specification ID: 12b83a17-24d8-5082-900f-0ab31fbfd3cb


#IBM AI Gov code to persist the auto-collected facts from main section

In [26]:
nb_name = "Credit Risk Demo (Scikit)"
nb_asset_id = "84ad277c-95dd-4f2c-aaaf-75ebec5fd6ad"
nb_asset_url = "https://" + CPD_URL + "/analytics/notebooks/v2/" + nb_asset_id + "?projectid=" + PROJECT_UID + "&context=cpdaas"

In [27]:
latestRunId = facts_client.runs.list_runs_by_experiment('1').sort_values('start_time').iloc[-1]['run_id']
facts_client.runs.log_metric(latestRunId, "Custom ROC AUC score", auc)
facts_client.runs.log_param(latestRunId, "Test data asset name", TEST_DATA_ASSSET_NAME)
facts_client.runs.log_param(latestRunId, "Test data asset id", test_data_asset_id)
facts_client.runs.log_param(latestRunId, "Test data asset URL", test_data_asset_url)
facts_client.runs.set_tags(latestRunId, {"Notebook name": nb_name, "Notebook id": nb_asset_id, "Notebook URL" : nb_asset_url})
facts_client.export_facts.export_payload(latestRunId)

2022/07/13 16:17:48 INFO : Initiating logging to factsheet for run_id......366f6be8e7e544909af699a5455a5236
2022/07/13 16:17:49 INFO : Successfully logged results to Factsheet service for run_id 366f6be8e7e544909af699a5455a5236 under asset_id: 3a652a16-5ee3-46ec-b5e0-a5458742c1f6 and project_id : 6d8022e5-71c7-41c7-a697-e6c26fabebeb


In [28]:
facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

{'name': 'Credit Risk Demo Model1 (SciKit)',
 'type': 'scikit-learn_1.0',
 'software_spec': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
 'training_data_references': [{'id': 'banking_demo_train.csv',
   'type': 'data_asset',
   'connection': {},
   'location': {'id': '104e0dda-d5ff-4edd-a4b3-bdf8fdb34dbb',
    'path': 'https://https://services-uscentral.skytap.com:20623/projects/6d8022e5-71c7-41c7-a697-e6c26fabebeb/data-assets/104e0dda-d5ff-4edd-a4b3-bdf8fdb34dbb/preview?context=cpdaas'},
   'schema': {'id': 'train',
    'fields': [{'name': 'CheckingStatus', 'type': 'string'},
     {'name': 'LoanDuration', 'type': 'integer'},
     {'name': 'CreditHistory', 'type': 'string'},
     {'name': 'LoanPurpose', 'type': 'string'},
     {'name': 'LoanAmount', 'type': 'integer'},
     {'name': 'ExistingSavings', 'type': 'string'},
     {'name': 'EmploymentDuration', 'type': 'string'},
     {'name': 'InstallmentPercent', 'type': 'integer'},
     {'name': 'Sex', 'type': 'string'},
     {'name': 'OthersO

# IBM WML code:  persisting model as an asset

In [29]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_target=['Risk'],
    training_data=train_data.drop('Risk', axis=1),
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: bfe21b7f-7e95-466c-ad73-fcfdcf09aa2a


#Deploy and score

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [30]:
wml_client.spaces.list(limit=10)

------------------------------------  --------------------------  ------------------------
ID                                    NAME                        CREATED
5e902387-2bab-4b6c-8a61-be637288dfcd  Banking-demo-production     2022-07-13T14:05:09.722Z
be528d28-0826-4f29-857b-c5e0eee69bc0  Banking-demo-preproduction  2022-07-13T14:03:46.522Z
81d2ff08-f228-404f-b925-453676d00567  Banking-demo-development    2022-07-13T14:01:08.128Z
------------------------------------  --------------------------  ------------------------


In [31]:
wml_client.set.default_project(PROJECT_UID)
wml_client.repository.promote_model(model_id='bfe21b7f-7e95-466c-ad73-fcfdcf09aa2a',source_project_id=PROJECT_UID,target_space_id='81d2ff08-f228-404f-b925-453676d00567')

'630352e8-8e01-43bc-85aa-5b685f6742e8'

In [32]:
wml_client.set.default_space("81d2ff08-f228-404f-b925-453676d00567")

Unsetting the project_id ...


'SUCCESS'

In [33]:
import json
print(json.dumps(wml_client.deployments.get_details(), indent=4))

{
    "resources": []
}


In [34]:
promoted_model_uid='630352e8-8e01-43bc-85aa-5b685f6742e8'
USE_EXISTING_DEPLOYMENT = False
if (USE_EXISTING_DEPLOYMENT) :
    deployment_uid = "4d670a85-c5df-4c24-b0ae-4a1ce2491527"
    #print(json.dumps(wml_client.deployments.get_details(deployment_uid), indent=4))
else: 
    print("Creating new Deployment for model...")
    metadata = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(promoted_model_uid, meta_props=metadata)
    deployment_uid = wml_client.deployments.get_uid(deployment)

    print("Model id: {}".format(model_uid))
    
print("Deployment id: {}".format(deployment_uid))
deployment_details = wml_client.deployments.get_details(deployment_uid);
space_uid = deployment_details["metadata"]["space_id"]
space_details = wml_client.spaces.get_details(space_uid)
print("Using Deployment: {} ({}) from space {} ({})".format(deployment_details["metadata"]["name"], space_uid, space_details["entity"]["name"], deployment_uid))

Creating new Deployment for model...


#######################################################################################

Synchronous deployment creation for uid: '630352e8-8e01-43bc-85aa-5b685f6742e8' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.....................................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='3fa4aa45-2a2b-480a-8bc4-a05bb264abb2'
------------------------------------------------------------------------------------------------


Model id: bfe21b7f-7e95-466c-ad73-fcfdcf09aa2a
Deployment id: 3fa4aa45-2a2b-480a-8bc4-a05bb264abb2
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
Using Deployment: Credit Risk 

#Score the model

In [35]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
        ]

scoring_payload = {"input_data": [{"fields": fields, "values": values}]}

In [36]:
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['No Risk', [0.6718286264691569, 0.3281713735308432]],
    ['No Risk', [0.7227741559454531, 0.27722584405454687]]]}]}

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.